In [1]:
# Import required libraries for data manipulation and analysis
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr

In [2]:
#Import required sklearn functions
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from collections import defaultdict
from sklearn.preprocessing import normalize

In [3]:
#Import sklearn classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
from sklearn.linear_model import LogisticRegression

In [4]:
#Import library to oversample 
from imblearn.over_sampling import RandomOverSampler

In [5]:
#Import RDKit and Mordred libraries
from rdkit import Chem
from rdkit.Chem import Draw
from mordred import Calculator, descriptors

In [6]:
#Allows figures to be visualized in jupyter notebook
%matplotlib inline

In [7]:
#Functions used in the study

#Remove those numbers from analysis data
def filter_rows_by_values1(df, col, values):
    return df[~df[col].isin(values)]

#Remove those numbers from analysis data
def filter_rows_by_values2(df, col, values):
    return df[df[col].isin(values)]

#Get Mordred calcs
def get_Mordred(data_input):
    # Assigns Reactants Mordred Info
    reactants = data_input['Substrate']
    
    reactants_mol_list = []
    for inChi_reactants in reactants:
      reactants_mol = Chem.MolFromInchi(inChi_reactants)
      reactants_mol_list.append(reactants_mol)

    # Puts reactants into Pandas Type
    reactant_data = []
    reactant_data = calc.pandas(reactants_mol_list)
       
    #Joins Mordred parameters with experimental, atomic charges, and JChem for Excel parameters
    add_reactants = pd.concat((data_input, reactant_data), axis=1)
    
    #Force any non-numeric entries as NaN and replace them with 0
    int_data = add_reactants.apply(pd.to_numeric, errors='coerce')
    
    output = int_data.fillna(0)#, inplace=True)

    return output

#Remove zero varience
def remove_zero_varience(values):
   sel = VarianceThreshold()
   _ = sel.fit(values)
   mask = sel.get_support()
   values = values.loc[:,mask] 
   return values

def remove_95correlated(correlated):
    #Remove any features that are greater than 95% correlated
    corr_matrix = correlated.corr()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]

    correlated = correlated.drop(to_drop, axis = 1)
    corr_matrix = correlated.corr()
    return correlated

def remove_nonimportant(X_values, y_values):
    # Specifys Random Forest and the Number of Trees, SelectFromModel will
    # select features which are most important
    feature_names = [f"feature {i}" for i in range(X_values.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(X_values, y_values)

    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time

    threshold = np.sort(importances)[-100]
    
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(X_values, y_values)

    # Select the final features set 
    sel.get_support()
    selected_feat= X_values.columns[(sel.get_support())]

    # Prints the names of the final selected features
    print(selected_feat)
    X_values = X_values[selected_feat]
    
    return X_values

def dendrogram(X_values, y):
    corr = spearmanr(X_values).correlation
    # Ensure the correlation matrix is symmetric
    corr = (corr + corr.T) / 2
    np.fill_diagonal(corr, 1)
    distance_matrix = 1 - np.abs(corr)
    dist_linkage = hierarchy.ward(squareform(distance_matrix))
  
    trained_cluster_ids = hierarchy.fcluster(dist_linkage, y, criterion="distance")
    trained_cluster_id_to_feature_ids = defaultdict(list) 
    for idx, trained_cluster_id in enumerate(trained_cluster_ids):
        trained_cluster_id_to_feature_ids[trained_cluster_id].append(idx)
    
    trained_selected_features = [v[0] for v in trained_cluster_id_to_feature_ids.values()]
    final_selected_features = X_values.columns[trained_selected_features]
    X_train = X_values[final_selected_features]
    return X_train

def classificationMetrics(results, y_test, pred):
    acc = accuracy_score(y_test, pred)
    prec = precision_score(y_test, pred, average=None, zero_division=0)
    recall = recall_score(y_test, pred, average=None)
    F1 = f1_score(y_test, pred, average=None)
    comb = np.concatenate((x, y, acc, prec, recall, F1), axis=None)
    comb = [comb]
    results = results.append(pd.DataFrame(comb, columns=results.columns), ignore_index=True)
    return results


In [8]:
# Sets Pandas Display to Monitor Code
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 15)

In [9]:
# Create Mordred Calculator
calc = Calculator(descriptors, ignore_3D=True)

In [10]:
# Read Training/Test data input File
data = pd.read_csv('BorylationTrainingTest 9-26-24.csv')
data.head()

,Substrate,Product,Boronic Ester,Active Catalyst-Ligand,Catalyst,Ligand,Buried_Vol,...,Platt Index,Refractivity,Ring Atom Count,Rot Bond Count,Sigma Electronegativity,Wiener Index,Product_Ratio
0,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-6-7-8-9-10-11-12-15-16-1...,2,6,1,1,43.1,...,12,38.6102,0,5,7.387931,84,1
1,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-7-8-9-10-11-12(2)15-16-1...,2,6,1,1,43.1,...,12,38.6102,0,5,7.470532,84,0
2,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-7-9-10-11-12(8-2)15-16-1...,2,6,1,1,43.1,...,12,38.6102,0,5,7.493894,84,0
3,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-7-9-11-12(10-8-2)15-16-1...,2,6,1,1,43.1,...,12,38.6102,0,5,7.496149,84,0
4,"InChI=1S/C7H15N/c1-2-8-6-4-3-5-7-8/h2-7H2,1H3","InChI=1S/C13H26BNO2/c1-12(2)13(3,4)17-14(16-12...",2,7,3,1,41.0,...,18,36.8787,6,1,7.937482,64,1


In [11]:
#group the compounds by numbers
data['grouped'] = data.groupby('Substrate', sort=False).ngroup()
data[['Substrate','grouped']]

,Substrate,grouped
0,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
1,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
2,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
3,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
4,"InChI=1S/C7H15N/c1-2-8-6-4-3-5-7-8/h2-7H2,1H3",1
...,...,...
1022,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,198
1023,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,198
1024,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,198
1025,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,198


In [12]:
#Convert substrates to Mordred features
data = get_Mordred(data)

100%|██████████| 1027/1027 [01:28<00:00, 11.65it/s]


In [13]:
MLPResults_df = pd.DataFrame(columns = ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

SVMResults_df = pd.DataFrame(columns =  ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

GaussResults_df = pd.DataFrame(columns =  ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

LRResults_df = pd.DataFrame(columns =  ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

MLPmaxacc_comb = pd.DataFrame()
SVMmaxacc_comb = pd.DataFrame()
Gaussmaxacc_comb = pd.DataFrame()
LRmaxacc_comb = pd.DataFrame()

model_columns = pd.DataFrame()
for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 198,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(data, "grouped", set_numbers)
    training_data = training_data.drop('grouped', axis = 1)    
    test_data = filter_rows_by_values2(data, "grouped", set_numbers)
    test_data = test_data.drop('grouped', axis = 1)   

    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop('Product_Ratio' , axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop('Product_Ratio' , axis = 1)
    test_y = test_data['Product_Ratio']    
    
    training_X = pd.DataFrame(
    normalize(training_X),
    columns = training_X.columns)
    
    #Remove features that dont change
    training_X = remove_zero_varience(training_X)
    
    #Remove features that are more than 95% correlated
    training_X = remove_95correlated(training_X)
     
    #Remove features that are considered less important
    feature_names = [f"feature {i}" for i in range(training_X.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(training_X, training_y)
    
    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time
    
    threshold = np.sort(importances)[-100] 
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(training_X, training_y)
     
    # Select the reduced features set 
    sel.get_support()
    selected_feat= training_X.columns[(sel.get_support())]
    
    reduced1_X = training_X[selected_feat]
    test_X = test_X[selected_feat]
    
    #Apply over-sampling to dataset
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(reduced1_X, training_y) 
    
    for y in [ 
              0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35,
              0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40,
              0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45,
              0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50,
             ]:
    
        #Make final training and test set and save them as df's  
        X_train = dendrogram(X_resampled, y)
        test_X = test_X[X_train.columns]
        training_columns_list = X_train.columns.tolist()
        training_columns_list = (x, y, training_columns_list)
        training_columns_list = pd.DataFrame(training_columns_list).T

        #MLP CLassifier
        MLP = MLPClassifier(alpha=1, max_iter=1000)
        MLP.fit(X_train, y_resampled)
        test_X_norm = pd.DataFrame(normalize(test_X), columns = test_X.columns)
        pred_MLP = MLP.predict(test_X_norm)
        MLPResults_df = classificationMetrics(MLPResults_df, test_y, pred_MLP)
        
        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = MLPResults_df.groupby('y')['acc'].mean()
        acc_std = MLPResults_df.groupby('y')['acc'].std()
        precision_0_mean = MLPResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = MLPResults_df.groupby('y')['precision 0'].std()        
        precision_1_mean = MLPResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = MLPResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = MLPResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = MLPResults_df.groupby('y')['recall 0'].std()      
        recall_1_mean = MLPResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = MLPResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = MLPResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = MLPResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = MLPResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = MLPResults_df.groupby('y')['F1 1'].std()
        
        MLPaverage_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        MLPaverage_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 
          
        
                
        MLPmaxacc = MLPaverage_df[MLPaverage_df.acc_mean == MLPaverage_df.acc_mean.max()]
        MLPmaxacc_copy  = MLPmaxacc.copy()
        MLPmaxacc_copy['x_col'] = x       
        
        #SVM CLassifier
        SVM = SVC(kernel="linear", C=1.0)
        SVM.fit(X_train, y_resampled)
        test_X_norm = pd.DataFrame(normalize(test_X), columns = test_X.columns)
        pred_SVM = SVM.predict(test_X_norm)
        SVMResults_df = classificationMetrics(SVMResults_df, test_y, pred_SVM)  
    
        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = SVMResults_df.groupby('y')['acc'].mean()
        acc_std = SVMResults_df.groupby('y')['acc'].std()
        precision_0_mean = SVMResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = SVMResults_df.groupby('y')['precision 0'].std()        
        precision_1_mean = SVMResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = SVMResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = SVMResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = SVMResults_df.groupby('y')['recall 0'].std()      
        recall_1_mean = SVMResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = SVMResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = SVMResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = SVMResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = SVMResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = SVMResults_df.groupby('y')['F1 1'].std()
        
        SVMaverage_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        SVMaverage_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 
                              
        SVMmaxacc = SVMaverage_df[SVMaverage_df.acc_mean == SVMaverage_df.acc_mean.max()]
        SVMmaxacc_copy  = SVMmaxacc.copy()
        SVMmaxacc_copy['x_col'] = x       
                
        #Gaussian Process CLassifier
        kernel =  ConstantKernel(constant_value=1,constant_value_bounds =(1e-5,1e6)) * RBF(length_scale=1e-1, length_scale_bounds=(1e-3, 1e3))
        Gauss = GaussianProcessClassifier(kernel = kernel)
        Gauss.fit(X_train, y_resampled)
        test_X_norm = pd.DataFrame(normalize(test_X), columns = test_X.columns)
        pred_Gauss = Gauss.predict(test_X_norm)
        GaussResults_df = classificationMetrics(GaussResults_df, test_y, pred_Gauss)
    
        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = GaussResults_df.groupby('y')['acc'].mean()
        acc_std = GaussResults_df.groupby('y')['acc'].std()
        precision_0_mean = GaussResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = GaussResults_df.groupby('y')['precision 0'].std()        
        precision_1_mean = GaussResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = GaussResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = GaussResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = GaussResults_df.groupby('y')['recall 0'].std()      
        recall_1_mean = GaussResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = GaussResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = GaussResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = GaussResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = GaussResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = GaussResults_df.groupby('y')['F1 1'].std()
        
        Gaussaverage_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        Gaussaverage_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 

        Gaussmaxacc = Gaussaverage_df[Gaussaverage_df.acc_mean == Gaussaverage_df.acc_mean.max()]
        Gaussmaxacc_copy  = Gaussmaxacc.copy()
        Gaussmaxacc_copy['x_col'] = x
        
        #Logistic CLassifier
        clf = LogisticRegression(solver = 'liblinear', C=0.5)
        clf.fit(X_train, y_resampled)
        test_X_norm = pd.DataFrame(normalize(test_X), columns = test_X.columns)
        pred_LR = clf.predict(test_X_norm)
        LRResults_df = classificationMetrics(LRResults_df, test_y, pred_LR)
        
        
        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = LRResults_df.groupby('y')['acc'].mean()
        acc_std = LRResults_df.groupby('y')['acc'].std()
        precision_0_mean = LRResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = LRResults_df.groupby('y')['precision 0'].std()        
        precision_1_mean = LRResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = LRResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = LRResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = LRResults_df.groupby('y')['recall 0'].std()      
        recall_1_mean = LRResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = LRResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = LRResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = LRResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = LRResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = LRResults_df.groupby('y')['F1 1'].std()
        
        LRaverage_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        LRaverage_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 

        LRmaxacc = LRaverage_df[LRaverage_df.acc_mean == LRaverage_df.acc_mean.max()]
        LRmaxacc_copy  = LRmaxacc.copy()
        LRmaxacc_copy['x_col'] = x                

    MLPmaxacc_comb = MLPmaxacc_comb.append(MLPmaxacc_copy)
    SVMmaxacc_comb = SVMmaxacc_comb.append(SVMmaxacc_copy)
    Gaussmaxacc_comb = Gaussmaxacc_comb.append(Gaussmaxacc_copy)
    LRmaxacc_comb = LRmaxacc_comb.append(LRmaxacc_copy)

#Write the results onto a CSV file, currently commented out 
MLPmaxacc_comb.to_csv("10Runs_ClassificationScreening_SVM_NN_GP_LR.csv",)
SVMmaxacc_comb.to_csv("10Runs_ClassificationScreening_SVM_NN_GP_LR.csv", mode="a")
Gaussmaxacc_comb.to_csv("10Runs_ClassificationScreening_SVM_NN_GP_LR.csv", mode="a")
LRmaxacc_comb.to_csv("10Runs_ClassificationScreening_SVM_NN_GP_LR.csv", mode="a")


In [14]:
MLPmaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.40,0.741899,0.011111,0.793159,0.001629,0.312524,0.011473,0.906429,0.021938,0.151282,0.028218,0.845890,0.008818,0.201963,0.023605,1
0.50,0.741899,0.005770,0.793496,0.000735,0.313048,0.007584,0.905714,0.010541,0.153846,0.012087,0.845872,0.004471,0.205921,0.009743,1
0.35,0.547702,0.248909,0.639684,0.328452,0.248567,0.078492,0.577406,0.419920,0.456410,0.398151,0.558002,0.383418,0.249148,0.087184,2
0.35,0.626219,0.231558,0.699705,0.279536,0.292685,0.096741,0.697742,0.382646,0.347863,0.360593,0.663420,0.345629,0.224874,0.085608,3
0.35,0.669913,0.213893,0.727438,0.245912,0.302014,0.121460,0.768986,0.352892,0.272436,0.338263,0.719656,0.313946,0.188406,0.100790,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.35,0.703973,0.182880,0.754840,0.203886,0.314185,0.101100,0.823371,0.300566,0.221368,0.292656,0.768174,0.265431,0.174077,0.105712,6
0.35,0.714283,0.171065,0.763474,0.189738,0.319082,0.094481,0.838929,0.280727,0.210256,0.272501,0.782672,0.248059,0.177583,0.099266,7
0.35,0.715143,0.160127,0.769993,0.178181,0.317397,0.088526,0.838896,0.262915,0.217628,0.256507,0.788104,0.232403,0.190500,0.099553,8


In [15]:
SVMmaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.35,0.659218,0.000000,0.796992,0.000000,0.260870,0.000000,0.757143,0.000000,0.307692,0.000000,0.776557,0.000000,0.282353,0.000000,1
0.40,0.659218,0.000000,0.796992,0.000000,0.260870,0.000000,0.757143,0.000000,0.307692,0.000000,0.776557,0.000000,0.282353,0.000000,1
0.45,0.659218,0.000000,0.796992,0.000000,0.260870,0.000000,0.757143,0.000000,0.307692,0.000000,0.776557,0.000000,0.282353,0.000000,1
0.50,0.659218,0.000000,0.796992,0.000000,0.260870,0.000000,0.757143,0.000000,0.307692,0.000000,0.776557,0.000000,0.282353,0.000000,1
0.35,0.431703,0.233425,0.398496,0.408849,0.232529,0.029077,0.378571,0.388406,0.653846,0.355146,0.388278,0.398365,0.310742,0.029126,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.50,0.676898,0.219659,0.669864,0.302422,0.195565,0.149580,0.785840,0.364732,0.235043,0.361208,0.721359,0.327976,0.132794,0.134978,6
0.50,0.694049,0.207480,0.689959,0.284011,0.228852,0.160860,0.812818,0.343773,0.212454,0.338627,0.744745,0.308719,0.132457,0.124817,7
0.35,0.683557,0.188404,0.711659,0.270881,0.315192,0.055184,0.781624,0.306418,0.285256,0.288898,0.743660,0.285268,0.227042,0.093256,8


In [16]:
Gaussmaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.35,0.776536,0.000000,0.784091,0.000000,0.333333,0.000000,0.985714,0.000000,0.025641,0.000000,0.873418,0.000000,0.047619,0.000000,1
0.40,0.776536,0.000000,0.784091,0.000000,0.333333,0.000000,0.985714,0.000000,0.025641,0.000000,0.873418,0.000000,0.047619,0.000000,1
0.45,0.776536,0.000000,0.784091,0.000000,0.333333,0.000000,0.985714,0.000000,0.025641,0.000000,0.873418,0.000000,0.047619,0.000000,1
0.50,0.776536,0.000000,0.784091,0.000000,0.333333,0.000000,0.985714,0.000000,0.025641,0.000000,0.873418,0.000000,0.047619,0.000000,1
0.40,0.786174,0.009888,0.789951,0.006013,0.166667,0.170996,0.992857,0.007328,0.012821,0.013154,0.879858,0.006607,0.023810,0.024428,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.50,0.724990,0.186171,0.804232,0.020847,0.170175,0.187747,0.877998,0.308534,0.131410,0.311359,0.790131,0.253350,0.063554,0.103519,8
0.40,0.729629,0.175954,0.811190,0.036615,0.236469,0.183709,0.882916,0.292638,0.136752,0.299468,0.797096,0.242969,0.085022,0.100335,9
0.45,0.729629,0.175954,0.804197,0.018752,0.236116,0.183787,0.883594,0.290714,0.133903,0.291411,0.798208,0.239809,0.084371,0.098738,9


In [17]:
LRmaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.35,0.720670,0.000000,0.796053,0.000000,0.296296,0.000000,0.864286,0.000000,0.205128,0.000000,0.828767,0.000000,0.242424,0.000000,1
0.40,0.720670,0.000000,0.796053,0.000000,0.296296,0.000000,0.864286,0.000000,0.205128,0.000000,0.828767,0.000000,0.242424,0.000000,1
0.45,0.720670,0.000000,0.796053,0.000000,0.296296,0.000000,0.864286,0.000000,0.205128,0.000000,0.828767,0.000000,0.242424,0.000000,1
0.50,0.720670,0.000000,0.796053,0.000000,0.296296,0.000000,0.864286,0.000000,0.205128,0.000000,0.828767,0.000000,0.242424,0.000000,1
0.35,0.470283,0.256892,0.798026,0.002025,0.250299,0.047193,0.445301,0.429870,0.589744,0.394607,0.439861,0.399009,0.290101,0.048915,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.50,0.686372,0.219585,0.670077,0.302546,0.201470,0.152760,0.801825,0.364584,0.222222,0.357564,0.729501,0.329758,0.133017,0.124748,6
0.50,0.701444,0.206427,0.690954,0.284467,0.232212,0.160332,0.823807,0.341465,0.208791,0.332297,0.751068,0.309535,0.142026,0.117476,7
0.35,0.685769,0.180987,0.812629,0.011164,0.309569,0.046239,0.783811,0.293510,0.288462,0.273908,0.750686,0.267638,0.237727,0.081620,8
